In [1]:
    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');

    var sparkConf = new SparkConf().setAppName("Example");
    var sc = new SparkContext(sparkConf);
    var SQLContext = require('eclairjs/sql/SQLContext');
    var MultilayerPerceptronClassifier = require("eclairjs/ml/classification/MultilayerPerceptronClassifier");
    var MulticlassClassificationEvaluator = require("eclairjs/ml/evaluation/MulticlassClassificationEvaluator");


In [2]:
    var sqlContext = new SQLContext(sc);

    var path = "examples/data/mllib/sample_multiclass_classification_data.txt";
    var dataFrame = sqlContext.read().format("libsvm").load(path);


 Split the data into train and test


In [3]:
    var splits = dataFrame.randomSplit([0.6, 0.4], 1234);
    var train = splits[0];
    var test = splits[1];


 specify layers for the neural network:
 input layer of size 4 (features), two intermediate of size 5 and 4
 and output of size 3 (classes)


In [4]:
    var layers = [4, 5, 4, 3];


 create the trainer and set its parameters


In [5]:
    var trainer = new MultilayerPerceptronClassifier()
        .setLayers(layers)
        .setBlockSize(128)
        .setSeed(1234)
        .setMaxIter(100);


 train the model


In [6]:
    var model = trainer.fit(train);


 compute precision on the test set


In [7]:
    var result = model.transform(test);
    var predictionAndLabels = result.select("prediction", "label");
    var evaluator = new MulticlassClassificationEvaluator()
        .setMetricName("precision");
var result = evaluator.evaluate(predictionAndLabels);
    print("Precision = " + result);

    sc.stop();
